In [1]:
import random
import operator
import matplotlib.pyplot as pl
import math
import time
import itertools

In [31]:
def fatorial(k):
    '''(int) -> int

    Recebe um inteiro k e retorna o valor de k!

    Pre-condicao: supoe que k eh um numero inteiro nao negativo.
    '''

    k_fat = 1
    cont = 1
    while cont < k:
        cont += 1       # o mesmo que cont = cont + 1
        k_fat *= cont   # o mesmo que k_fat = k_fat * cont

    return k_fat

def combinacao(m, n):
    '''(int, int) -> int

    Recebe dois inteiros m e n, e retorna o valor de m!/((m-n)! n!)
    '''

    return fatorial(m)/(fatorial(n)*fatorial(m-n))


def number_of_colors(G):   #número de cores que será utilizado para colorir o grafo, definimos como 2 vezes o grau máximo
    max_degree = max(G.degree())
    return max_degree * 2


def energy(G):  #calcula o número de colisões, ou seja, a quantidade de vezes em que duas arestas
                #distintas adjacentes foram coliridas com a mesma cor
    
    ones = 0
    neighbors = []
    edges_seen = []
    
    for edge in G.edges():
        neighbors = []
        v1, v2 = edge[0] , edge[1]
        if (v1,v2) not in edges_seen and (v2,v1) not in edges_seen:
            for element in G.edges_incident(v1):
                if element[1] != v2:
                    neighbors.append(element)
            for element in G.edges_incident(v2):
                if element[0] != v1:
                    neighbors.append(element)
            for element in neighbors:
                if edge[2] == element[2]:
                    ones += 1
            edges_seen.append((v1,v2))
        #print(neighbors, edge[0], edge[1])
    return ones / 2
        
        
def random_color(G):  #colori de forma aleatória as arestas do grafo
    k = number_of_colors(G) 
    for element in G.edges():
        choice = random.randint(0,number_of_colors(G)-1)
        G.set_edge_label(element[0],element[1],choice)
    
    
def transition(edge_atual,current): #escolhe uma aresta adjacente para transitar
    edges_seen = []
    neighbors = []
    v1, v2 = edge_atual[0] , edge_atual[1]
    if (v1,v2) not in edges_seen and (v2,v1) not in edges_seen:
            for element in current.edges_incident(v1):
                if element[1] != v2:
                    neighbors.append(element)
            for element in current.edges_incident(v2):
                if element[0] != v1:
                    neighbors.append(element)
    choice = random.choice(neighbors) #escolhe um vizinho aleatório

    return choice


def acceptance_probability(curr_energy, candidate_energy,temp):
    '''
    Acceptance probability using boltzmann:
    '''
    
    return math.exp(-abs(candidate_energy - curr_energy) / temp)



def annealing(G, temp, stopping_iteration, alpha):
    H = Graph()
    start = time.time()
    random_color(G)
    colors = number_of_colors(G)
    initial_energy = energy(G)
    print('energia inicial {}'.format(initial_energy))
    current = G.copy()
    current_energy = initial_energy
    current_edge = random.choice(G.edges())
    best_solution = Graph()
    G.show(color_by_label = True)

    combination_of_colors = combinacao(colors,2)
    stopping_temp = 0.0001
    iteration = 0
    alpha = 0.7
    
    best_solution_energy = 10000
    
    while temp >= stopping_temp: 
        while iteration < stopping_iteration:
    
            #transição
            
            new_edge = transition(current_edge,current)
            new_color = random.randint(0,colors - 1)  #Atribuí uma cor ao vizinho de forma aleatória
            candidate = current.copy() #cópia do grafo atual
            candidate.set_edge_label(new_edge[0],new_edge[1],new_color) #modifica a cor da aresta, transitando. 
            candidate_energy = energy(candidate)
            

            #aceitação
            forests = -1
            if candidate_energy <= current_energy: #aceitamos e atualizamos o grafo atual
                if candidate_energy == 0:
                    forests = 0
                    colors_seen = []
                    for i in range(colors):
                        for j in range(1, colors):
                            if (i,j) not in colors_seen and (j,i) not in colors_seen and i != j:
                                K = subgraph_by_two_colors(i,j,candidate)
                                colors_seen.append((i,j))
                                if K.is_forest():
                                    forests += 1
                    if forests == combination_of_colors:
                        best_solution = candidate.copy()
                        best_solution_energy = energy(best_solution)
                        break
                current = candidate.copy()
                current_energy = candidate_energy
                current_edge = new_edge
            else:
                unif = random.random() # generate a uniform number between 0 and 1
                if unif < acceptance_probability(current_energy, candidate_energy,temp):
                    if candidate_energy == 0:
                        forests = 0
                        colors_seen = []
                        for i in range(colors):
                            for j in range(1, colors):
                                if (i,j) not in colors_seen and (j,i) not in colors_seen and i != j:
                                    K = subgraph_by_two_colors(i,j,candidate)
                                    colors_seen.append((i,j))
                                    if K.is_forest():
                                        forests += 1
                        if forests == combination_of_colors:
                            best_solution = candidate.copy()
                            best_solution_energy = energy(best_solution)
                            break
                    current = candidate.copy()
                    current_energy = candidate_energy
                    current_edge = new_edge
            
            iteration += 1

        if forests == combination_of_colors:
            best_solution.show(color_by_label = True)
            stop = time.time()
            print("The time of the run:", stop - start)
            H = best_solution
            return best_solution_energy, H
        
        temp = temp * alpha
        iteration = 0
    print(current_energy)
    H = current
    return current_energy, H

def subgraph_by_two_colors(color1, color2, G):
    H = Graph()
    for e in G.edges():
        if e[2] == color1:
            H.add_edge(e)
        if e[2] == color2:
            H.add_edge(e)
    return H

In [ ]:
start = time.time()
teste = graphs.RandomGNP(20,1)
iteration = len(teste.edges())
if teste.is_connected():
    k = 1
    while k != 0:
        k, H = annealing(teste,temp = 10000000, stopping_iteration = 200, alpha = 0.95)  

stop = time.time()
print("The time of the run:", stop - start)

In [ ]:
forests = 0
colors_seen = []
colors = number_of_colors(H)
for i in range(colors):
    for j in range(1, colors):
        if (i,j) not in colors_seen and (j,i) not in colors_seen and i != j:
            print(i,j)
            K = subgraph_by_two_colors(i,j,H)
            colors_seen.append((i,j))
            K.show(color_by_label=True)
            if K.is_forest():
                forests += 1
if forests == combinacao(colors,2):
    print('yesss')
else:
    print('nooo')